#### NOTA: En este cuaderno se crearan las Tablas `empleado`, `cargo` y `sector` del Datawarehouse

***
### Importando Librerias

In [1]:
from pyspark.sql import SparkSession

In [2]:
from pyspark.sql.functions import col, max, count, when, isnull, monotonically_increasing_id

***
Creando Session de Spark

In [3]:
spark = SparkSession.builder.appName('empleado-cargo-sector').getOrCreate()

***
Importando Conexion a Contenedor `mod4_mysql`

In [4]:
from db.connection import connec_db

***
Trabajando el csv asociado a  `empleado`

In [4]:
empleado = spark.read.csv("Data/Empleados.csv", sep=",", header=True, inferSchema=True)

***
Visualizando como vienen los Datos

In [5]:
empleado.show(5)

+-----------+---------+---------+--------+--------------+--------------+-------+
|ID_empleado| Apellido|   Nombre|Sucursal|        Sector|         Cargo|Salario|
+-----------+---------+---------+--------+--------------+--------------+-------+
|       1968|   Burgos| Jeronimo| Caseros|Administración|Administrativo|32000.0|
|       1674| Villegas|Estefania| Caseros|Administración|      Vendedor|32000.0|
|       1516|Fernandez|Guillermo| Caseros|Administración|      Vendedor|45000.0|
|       1330|  Ramirez|   Eliana| Caseros|Administración|      Vendedor|32000.0|
|       1657|  Carmona|     Jose| Caseros|Administración|      Vendedor|32000.0|
+-----------+---------+---------+--------+--------------+--------------+-------+
only showing top 5 rows



***
Visualizando la Inferencia del Schema

In [6]:
empleado.printSchema()

root
 |-- ID_empleado: integer (nullable = true)
 |-- Apellido: string (nullable = true)
 |-- Nombre: string (nullable = true)
 |-- Sucursal: string (nullable = true)
 |-- Sector: string (nullable = true)
 |-- Cargo: string (nullable = true)
 |-- Salario: double (nullable = true)



***
Buscando Valores Faltantes

In [7]:
# Contar valores nulos por columna
empleado.agg(*[count(when(isnull(c), c)).alias(c) for c in empleado.columns]).show()

+-----------+--------+------+--------+------+-----+-------+
|ID_empleado|Apellido|Nombre|Sucursal|Sector|Cargo|Salario|
+-----------+--------+------+--------+------+-----+-------+
|          0|       0|     0|       0|     0|    0|      0|
+-----------+--------+------+--------+------+-----+-------+



Renombrando la columna 'ID_empleado'

In [8]:
empleado = empleado.withColumnRenamed("ID_empleado", "IdEmpleado")
empleado.show(1)

+----------+--------+--------+--------+--------------+--------------+-------+
|IdEmpleado|Apellido|  Nombre|Sucursal|        Sector|         Cargo|Salario|
+----------+--------+--------+--------+--------------+--------------+-------+
|      1968|  Burgos|Jeronimo| Caseros|Administración|Administrativo|32000.0|
+----------+--------+--------+--------+--------------+--------------+-------+
only showing top 1 row



***
Creando el Dataframe que representa la tabla `cargo`

In [9]:
cargo = empleado.select("Cargo").distinct()

In [10]:
cargo = cargo.withColumn("IdCargo", monotonically_increasing_id()+1)

In [11]:
cargo.show()

+-------------------+-------+
|              Cargo|IdCargo|
+-------------------+-------+
|            Técnico|      1|
|          Vendedor |      2|
|Aux. Administrativo|      3|
|           Vendedor|      4|
|     Administrativo|      5|
|       Aux. Técnico|      6|
+-------------------+-------+



***
Creando el Dataframe que representa la tabla `sector`

In [12]:
sector = empleado.select("Sector").distinct()

In [13]:
sector = sector.withColumn("IdSector", monotonically_increasing_id()+1)

In [14]:
sector.show()

+--------------+--------+
|        Sector|IdSector|
+--------------+--------+
|        Diseño|       1|
|  Comunicación|       2|
|Administración|       3|
|        Ventas|       4|
|       Derecho|       5|
|    Publicidad|       6|
+--------------+--------+



***
Mapeando Sector en la tabla `empleado` utilizando join

In [15]:
map_sector = sector.withColumnRenamed("Sector", "Sector_map")

In [16]:
empleado_join_sector = empleado.join(map_sector, map_sector["Sector_map"] == empleado["Sector"])
empleado_join_sector.show(1)

+----------+--------+--------+--------+--------------+--------------+-------+--------------+--------+
|IdEmpleado|Apellido|  Nombre|Sucursal|        Sector|         Cargo|Salario|    Sector_map|IdSector|
+----------+--------+--------+--------+--------------+--------------+-------+--------------+--------+
|      1968|  Burgos|Jeronimo| Caseros|Administración|Administrativo|32000.0|Administración|       3|
+----------+--------+--------+--------+--------------+--------------+-------+--------------+--------+
only showing top 1 row



In [17]:
# Verificando que el Join se Hizo correctamente Sin Nulos
empleado_join_sector.agg(*[count(when(isnull(c), c)).alias(c) for c in empleado_join_sector.columns]).show()

+----------+--------+------+--------+------+-----+-------+----------+--------+
|IdEmpleado|Apellido|Nombre|Sucursal|Sector|Cargo|Salario|Sector_map|IdSector|
+----------+--------+------+--------+------+-----+-------+----------+--------+
|         0|       0|     0|       0|     0|    0|      0|         0|       0|
+----------+--------+------+--------+------+-----+-------+----------+--------+



In [18]:
# Descartando la columna Sector 
empleado = empleado_join_sector.select("IdEmpleado", "Apellido", "Nombre", "Sucursal", "Cargo", "IdSector", "Salario")

In [19]:
del map_sector
del empleado_join_sector

***
Mapeando Cargo en la tabla `empleado` utilizando join

In [20]:
map_cargo = cargo.withColumnRenamed("Cargo", "Cargo_map")

In [21]:
empleado_join_cargo = empleado.join(map_cargo, map_cargo["Cargo_map"] == empleado["cargo"])

In [22]:
# Verificando que el Join se Hizo correctamente Sin Nulos
empleado_join_cargo.agg(*[count(when(isnull(c), c)).alias(c) for c in empleado_join_cargo.columns]).show()

+----------+--------+------+--------+-----+--------+-------+---------+-------+
|IdEmpleado|Apellido|Nombre|Sucursal|Cargo|IdSector|Salario|Cargo_map|IdCargo|
+----------+--------+------+--------+-----+--------+-------+---------+-------+
|         0|       0|     0|       0|    0|       0|      0|        0|      0|
+----------+--------+------+--------+-----+--------+-------+---------+-------+



In [23]:
# Descartando la columna Cargo
empleado = empleado_join_cargo.select("IdEmpleado","Apellido","Nombre","Sucursal","IdSector","IdCargo","Salario")


In [24]:
del map_cargo
del empleado_join_cargo

Se debe Normalizar la Columna `sucursal` para poder mapearla

In [25]:
empleado = empleado.withColumn('Sucursal', when(col('Sucursal')\
                               .isin(["Mendoza1"]), 'Mendoza-1')\
                               .otherwise(col('Sucursal')))

empleado = empleado.withColumn('Sucursal', when(col('Sucursal')\
                               .isin(["Mendoza 1"]), 'Mendoza-1')\
                               .otherwise(col('Sucursal')))

empleado = empleado.withColumn('Sucursal', when(col('Sucursal')\
                               .isin(["Mendoza2"]), 'Mendoza-2')\
                               .otherwise(col('Sucursal')))

empleado = empleado.withColumn('Sucursal', when(col('Sucursal')\
                               .isin(["Mendoza 2"]), 'Mendoza-2')\
                               .otherwise(col('Sucursal')))


empleado = empleado.withColumn('Sucursal', when(col('Sucursal')\
                               .isin(["MDQ1"]), 'Mdq-1')\
                               .otherwise(col('Sucursal')))

empleado = empleado.withColumn('Sucursal', when(col('Sucursal')\
                               .isin(["MDQ2"]), 'Mdq-2')\
                               .otherwise(col('Sucursal')))

empleado = empleado.withColumn('Sucursal', when(col('Sucursal')\
                               .isin(["Rosario1"]), 'Rosario-1')\
                               .otherwise(col('Sucursal')))

empleado = empleado.withColumn('Sucursal', when(col('Sucursal')\
                               .isin(["Rosario2"]), 'Rosario-2')\
                               .otherwise(col('Sucursal')))

empleado = empleado.withColumn('Sucursal', when(col('Sucursal')\
                               .isin(["Palermo 1"]), 'Palermo-1')\
                               .otherwise(col('Sucursal')))

empleado = empleado.withColumn('Sucursal', when(col('Sucursal')\
                               .isin(["Palermo 2"]), 'Palermo-2')\
                               .otherwise(col('Sucursal')))

empleado = empleado.withColumn('Sucursal', when(col('Sucursal')\
                               .isin(["Cordoba Quiroz"]), 'Córdoba Quiroz')\
                               .otherwise(col('Sucursal')))

In [26]:
empleado.select("Sucursal").distinct().show(31)

+------------------+
|          Sucursal|
+------------------+
|Cerro de las Rosas|
|         Rosario-1|
|    Córdoba Quiroz|
|          Deposito|
|             Mdq-1|
|             Mdq-2|
|    Córdoba Centro|
|           Almagro|
|            Flores|
|         Bariloche|
|         Palermo-2|
|     Vicente Lopez|
|             Velez|
|           Alberdi|
|         Palermo-1|
|        Corrientes|
|        Avellaneda|
|          La Plata|
|        San Isidro|
|             Moron|
|         Caballito|
|         Mendoza-2|
|           Cabildo|
|         Rosario-2|
|          Castelar|
|           Quilmes|
|           Caseros|
|             Lanus|
|         San Justo|
|         Mendoza-1|
+------------------+



***
Para poder mapear la columna 'sucursal' dentro de `empleado` necesito traer la informacion de <br>
la tabla `sucursal` ya normalizada. Cuando se trabajo 'sucursal' se dejo un archivo csv llamado <br>
'sucursal_normalizada.csv` 

In [27]:
sucursal = spark.read.csv("Data/sucursal_normalizada.csv", sep=",", header=True ,inferSchema=True)

In [28]:
sucursal.show(1)

+----------+--------+-------------+-----------+----------+---------+
|IdSucursal|Sucursal|    Domicilio|IdLocalidad|   Latitud| Longitud|
+----------+--------+-------------+-----------+----------+---------+
|        15|Castelar|Sta Rosa 1845|          9|-34.639305|-58.65567|
+----------+--------+-------------+-----------+----------+---------+
only showing top 1 row



In [29]:
sucursal_map = sucursal.select("IdSucursal", "Sucursal")
sucursal_map = sucursal_map.withColumnRenamed("Sucursal", "Sucur_map")

In [30]:
sucursal_map.show(1)

+----------+---------+
|IdSucursal|Sucur_map|
+----------+---------+
|        15| Castelar|
+----------+---------+
only showing top 1 row



In [31]:
del sucursal

Mapeando Sucursal en la tabla `empleado` utilizando join

In [32]:
empleado_join_sucursal = empleado.join(sucursal_map, sucursal_map["Sucur_map"] == empleado["Sucursal"])

In [33]:
# Verificando que el Join se Hizo correctamente Sin Nulos
empleado_join_sucursal.agg(*[count(when(isnull(c), c)).alias(c) for c in empleado_join_sucursal.columns]).show()

+----------+--------+------+--------+--------+-------+-------+----------+---------+
|IdEmpleado|Apellido|Nombre|Sucursal|IdSector|IdCargo|Salario|IdSucursal|Sucur_map|
+----------+--------+------+--------+--------+-------+-------+----------+---------+
|         0|       0|     0|       0|       0|      0|      0|         0|        0|
+----------+--------+------+--------+--------+-------+-------+----------+---------+



In [34]:
# Descartando la Columna Sucursal
empleado = empleado_join_sucursal.select("IdEmpleado", "Apellido", "Nombre", "IdSector",
                                         "IdSucursal", "IdCargo", "Salario")

In [35]:
empleado.count()

267

In [36]:
del empleado_join_sucursal
del sucursal_map

***
#### Tratamiento de Valores Repetidos en `IdEmpleado`
Creo la columna "CodigoEmpleado" y le asigno el mismo valor que hasta ahora tiene "IdEmpleado"

In [37]:
empleado = empleado.withColumn("CodigoEmpleado", col("IdEmpleado"))
empleado.show(10)

+----------+---------+---------+--------+----------+-------+-------+--------------+
|IdEmpleado| Apellido|   Nombre|IdSector|IdSucursal|IdCargo|Salario|CodigoEmpleado|
+----------+---------+---------+--------+----------+-------+-------+--------------+
|      1968|   Burgos| Jeronimo|       3|        13|      5|32000.0|          1968|
|      1674| Villegas|Estefania|       3|        13|      4|32000.0|          1674|
|      1516|Fernandez|Guillermo|       3|        13|      4|45000.0|          1516|
|      1330|  Ramirez|   Eliana|       3|        13|      4|32000.0|          1330|
|      1657|  Carmona|     Jose|       3|        13|      4|32000.0|          1657|
|      1573|De santis|  Marcela|       3|        13|      3|15000.0|          1573|
|      1658|   Franco|  Daniela|       3|        13|      4|32000.0|          1658|
|      1078|   Cortes|   Rafael|       1|        13|      5|42000.0|          1078|
|      1695|   Berrio|   Camilo|       1|         1|      4|32000.0|        

***
En la Clase se Establecio que el nuevo `IdEmpleado` seria (sucursal * 1.000.000) + IdEmpleado_viejo

In [38]:
# Borro la columna `IdEmpleado`, la informacion ya la tengo en `CodigoEmpleado`
empleado = empleado.drop('IdEmpleado')

In [39]:
# Creo nuevamente la columna `IdEmpleado` pero esta vez (sucursal * 1.000.000) + CodigoEmpleado
empleado = empleado.withColumn("IdEmpleado", (col("IdSucursal") * 1000000) + col("CodigoEmpleado"))

In [40]:
empleado.show()

+-----------+---------+--------+----------+-------+-------+--------------+----------+
|   Apellido|   Nombre|IdSector|IdSucursal|IdCargo|Salario|CodigoEmpleado|IdEmpleado|
+-----------+---------+--------+----------+-------+-------+--------------+----------+
|     Burgos| Jeronimo|       3|        13|      5|32000.0|          1968|  13001968|
|   Villegas|Estefania|       3|        13|      4|32000.0|          1674|  13001674|
|  Fernandez|Guillermo|       3|        13|      4|45000.0|          1516|  13001516|
|    Ramirez|   Eliana|       3|        13|      4|32000.0|          1330|  13001330|
|    Carmona|     Jose|       3|        13|      4|32000.0|          1657|  13001657|
|  De santis|  Marcela|       3|        13|      3|15000.0|          1573|  13001573|
|     Franco|  Daniela|       3|        13|      4|32000.0|          1658|  13001658|
|     Cortes|   Rafael|       1|        13|      5|42000.0|          1078|  13001078|
|     Berrio|   Camilo|       1|         1|      4|320

***
Creando la Conexion

In [39]:
engine = connec_db()

***
Pasando el Dataframe a Dataframe de Pandas  y vaciando en su respectiva Tabla

In [41]:
# IMPORTANDO LIBRERIA PANDAS
import pandas as pd

***
Vaciando el Dataframe a la Tabla `cargo`

In [42]:
pandas_df_cargo = cargo.toPandas()


In [42]:
pandas_df_cargo.to_sql(name="cargo", con=engine, index=False, if_exists="append")

6

In [43]:
pandas_df_cargo.to_csv("csv_practica/cargo.csv")

In [44]:
del cargo
del pandas_df_cargo

***
Vaciando el Dataframe a la Tabla `sector`

In [45]:
pandas_df_sector = sector.toPandas()

In [45]:
pandas_df_sector.to_sql(name="sector", con=engine, index=False, if_exists="append")

6

In [46]:
pandas_df_sector.to_csv("csv_practica/sector.csv", index=False)

In [47]:
del sector
del pandas_df_sector

***
Vaciando el Dataframe a la Tabla `empleado`

In [48]:
pandas_df_empleado = empleado.toPandas()

In [48]:
pandas_df_empleado.to_sql(name="empleado", con=engine, index=False, if_exists="append")

267

In [49]:
pandas_df_empleado.to_csv("csv_practica/empleado.csv", index=False)

In [50]:
del empleado

Guardo la informacion de empleado ya normalizado en un csv por si es necesario cuando se trabaje `venta`

In [46]:
pandas_df_empleado.to_csv("Data/empleado_normalizado.csv")

In [51]:
del pandas_df_empleado

In [52]:
spark.stop()